In [13]:
#pip install -qU langchain-openai

In [14]:
#pip install -U langchain-community faiss-cpu


In [15]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, LLMChain
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.agents import AgentExecutor, AgentType, ConversationalChatAgent

In [16]:
import os
from dotenv import load_dotenv

load_dotenv("OpenAl.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

In [17]:
path_pkl = 'my_faiss_index/index.pkl'
path_faiss = 'my_faiss_index/index.faiss'
faiss_path = os.path.join("my_faiss_index")

In [18]:

# Create a prompt template with ALL required variables
prompt_template = """You are a helpful Medical assistant for question-answering tasks. Use the following pieces of retreived context
     context to answer the user's question. 

    You have access to the following tools: {tool_names}
    {tools}
    
    Chat History:
    
    {chat_history}

    Use these appropriate tools to answer the user's question

    Remember:
    1. First consider if you need to use a tool or can answer directly
    2. If you need specialized medical knowledge, use the MedicalKnowledgeBase tool
    3. Format your response in a friendly, professional manner
    4. If you don't know the answer, say so clearly rather than making up information

Use the following format:

Thought: The user is asking about {input}. 
Action: Call {tool_names} with the input {input}
Action Input: {input}
Observation: Here’s the result of the action.
Thought: Based on this observation, I now know the final answer.
Final Answer: {final_answer}

"""

prompt = PromptTemplate.from_template(prompt_template)

In [19]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [20]:

# Assuming your FAISS vector store is initialized
embeddings = OpenAIEmbeddings()

def vector_retrieval(query: str):
    faiss_vectorstore = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
    retriever = faiss_vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 2})
    docs = retriever.invoke(query)
    
    return "\n".join([doc.page_content for doc in docs])

#Load our responses
faiss_tool = Tool(
    func= vector_retrieval,
    name="MedicalKnowledgeBase",
    description="Use this to answer questions about medical conditions. Input should be a specific medical_related question.."
)


tools = [faiss_tool]

In [21]:
agent = ConversationalChatAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True
)

In [22]:

print("💬 Medical Assistant Chatbot (type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye, thanks for reaching out to us!")
        break   
    try:
        # Use the agent to get a response - pass input directly as a string
        response = agent_executor.invoke({"input": user_input})
        # Print the response
        print(f"Medical Assistant: {response['output']}")
    except Exception as e:
        print(f"Error: {e}")
        print("Medical Assistant: I'm sorry, I encountered an error. Let me try again.")

💬 Medical Assistant Chatbot (type 'exit' to quit)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you today?"
}
```

> Finished chain.
Medical Assistant: Hello! How can I assist you today?


> Entering new AgentExecutor chain...
```json
{
    "action": "MedicalKnowledgeBase",
    "action_input": "What is a cyst and how is it treated?"
}
```
Observation: Hello. What are the symptoms in it? How long has it been there?
Hi. For further doubts consult a sexologist online -->
"Hi, Welcome to iclinq.com.  If its appearance is like curd, then probably it is yeast and you can take a single dose of tablet Canestein 150 mg to treat it."
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The response to your last comment was about the appearance of a cyst and the suggestion to take a tablet Canestein 150 mg if it appears like curd."
}
```

> Finished chain.
Medical Assistant: The response to your last c

In [23]:
#Creat a simple gradio interface for the chatbot

